# VIRTUALS PROTOCOL DYNAMIC METRICS

To utilise this notebook correctly, the "user_wallet", "agent_token", "liquidity_in", "buyback_tokens" and "sizing" variables must have their values adjusted appropriately.

Using values provided by Virtuals, for their buyback programme (https://dune.com/queries/4598245), this book is able to calculate maximal profit potential for the agents.

The thesis is simple: The optimal way to play the programme is to target expected volume on the daily timeframe. We can identify a safe-range to buy, using the average daily buyback volume. From this, we can construct a conservative profitability thesis and execute.

DISCLAIMER: This book is basic and should not be used (in isolation) to develop theses for trading the Buyback Programme. It serves as a foundation, and hint, for those looking to develop structured models.

In [ ]:
base_rpc = "https://base.llamarpc.com"

user_wallet = "" # YOUR WALLET ADDRESS

weth_token = "0x4200000000000000000000000000000000000006"
usdc_token = "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913"
agent_token = "0x98f4779FcCb177A6D856dd1DfD78cd15B7cd2af5" # AGENT TO ANALYZE
virtual_token = "0x0b3e328455c4059EEb9e3f84b5543F74E24e7E1b"
uniswap_router = "0x4752ba5dbc23f44d87826276bf6fd6b1c372ad24"
uniswap_factory = "0x8909Dc15e40173Ff4699343b6eB8132c65e18eC6"

In [ ]:
import json

with open("./abi/ERC20.json", "r") as file:
    erc20_abi = json.load(file)

with open("./abi/UniswapPool.json", "r") as file:
    pool_abi = json.load(file)

with open("./abi/UniswapRouter.json", "r") as file:
    router_abi = json.load(file)

with open("./abi/UniswapFactory.json", "r") as file:
    factory_abi = json.load(file)

In [ ]:
from web3 import Web3

web3 = Web3(Web3.HTTPProvider(base_rpc))

eth_balance = web3.eth.get_balance(user_wallet)

virtual_contract = web3.eth.contract(address=virtual_token, abi=erc20_abi)
agent_contract = web3.eth.contract(address=agent_token, abi=erc20_abi)

virtual_balance = virtual_contract.functions.balanceOf(user_wallet).call()
agent_balance = agent_contract.functions.balanceOf(user_wallet).call()

agent_ticker = agent_contract.functions.symbol().call()

print("Wallet Balances:")
print(round(web3.from_wei(eth_balance, 'ether'), 3), "ETH")
print(round(web3.from_wei(virtual_balance, 'ether'), 3), "VIRTUAL")
print(round(web3.from_wei(agent_balance, 'ether'), 3), agent_ticker)

In [ ]:
weth_contract = web3.eth.contract(address=weth_token, abi=erc20_abi)
usdc_contract = web3.eth.contract(address=usdc_token, abi=erc20_abi)

factory_contract = web3.eth.contract(address=uniswap_factory, abi=factory_abi)
liquidity_pool = factory_contract.functions.getPair(weth_token, usdc_token).call()
pool_contract = web3.eth.contract(address=liquidity_pool, abi=pool_abi)

pool_reserves = pool_contract.functions.getReserves().call()

supply_tokens = pool_reserves[0] / 10 ** weth_contract.functions.decimals().call()
liquidity_tokens = pool_reserves[1] / 10 ** usdc_contract.functions.decimals().call()

weth_price = liquidity_tokens / supply_tokens

print("WETH QUANTITY: ", round(supply_tokens, 3), "WETH")
print("USDC QUANTITY: ", round(liquidity_tokens, 3), "USDC")
print("WETH PRICE: $", round(weth_price, 3))

In [ ]:
factory_contract = web3.eth.contract(address=uniswap_factory, abi=factory_abi)

liquidity_pool = factory_contract.functions.getPair(virtual_token, weth_token).call()
pool_contract = web3.eth.contract(address=liquidity_pool, abi=pool_abi)

pool_reserves = pool_contract.functions.getReserves().call()

supply_tokens = pool_reserves[0] / 10 ** weth_contract.functions.decimals().call()
liquidity_tokens = pool_reserves[1] / 10 ** virtual_contract.functions.decimals().call()

virtuals_price = (liquidity_tokens / supply_tokens) * weth_price

print("WETH QUANTITY: ", round(supply_tokens, 3), "WETH")
print("VIRTUAL QUANTITY: ", round(liquidity_tokens, 3), "VIRTUAL")
print("VIRTUALS PRICE: $", round(virtuals_price, 3))

In [ ]:
liquidity_pool = factory_contract.functions.getPair(agent_token, virtual_token).call()
pool_contract = web3.eth.contract(address=liquidity_pool, abi=pool_abi)

total_supply = agent_contract.functions.totalSupply().call() / 10 ** agent_contract.functions.decimals().call()
pool_reserves = pool_contract.functions.getReserves().call()

supply_tokens = pool_reserves[1] / 10 ** agent_contract.functions.decimals().call()
liquidity_tokens = pool_reserves[0] / 10 ** virtual_contract.functions.decimals().call()

constant_product = supply_tokens * liquidity_tokens

token_price = (liquidity_tokens / supply_tokens) * virtuals_price
market_cap = token_price * total_supply

print("VIRTUAL QUANTITY: ", round(liquidity_tokens, 3), "VIRTUAL")
print("AGENT QUANTITY: ", round(supply_tokens, 3), agent_ticker)
print("AGENT PRICE: $", token_price)
print("MARKET CAP: $", round(market_cap, 3))

In [ ]:
liquidity_in = 250  # BUY

liquidity_tokens += liquidity_in

supply_out = supply_tokens - (constant_product / liquidity_tokens)
supply_tokens = constant_product / liquidity_tokens

token_price = (liquidity_tokens / supply_tokens) * virtuals_price
market_cap = token_price * total_supply

print("NEW TOKEN PRICE: $", token_price)
print("NEW MARKET CAP: $", round(market_cap, 3))
print()
print("NEW LIQUIDITY: ", round(liquidity_tokens, 3), "VIRTUAL")
print("NEW LIQUIDITY (USD): $", round(liquidity_tokens * virtuals_price, 3))
print()
print("TOKENS RECEIVED: ", round(supply_out, 3), agent_ticker)

In [ ]:
buyback_tokens = 690344

market_cap_old = market_cap

In [ ]:
daily_buys = buyback_tokens / 30

liquidity_buyback_ratio = liquidity_tokens / daily_buys

liquidity_tokens += daily_buys
supply_tokens = constant_product / liquidity_tokens

token_price = (liquidity_tokens / supply_tokens) * virtuals_price
market_cap = token_price * total_supply

profitability = market_cap / market_cap_old

print("NEW TOKEN PRICE (DAY): $", token_price)
print("NEW MARKET CAP (DAY): $", round(market_cap, 3))
print()
print("PROFITABILITY: ", round(profitability, 3))
print()
print("LP/BUYBACK RATIO: ", round(liquidity_buyback_ratio, 3))
print("DAILY BUYS: $", round(daily_buys * virtuals_price, 3))

In [ ]:
sizing = 250 # LATEST (FUTURE?) POSITION SIZE - EXCLUDING CURRENT BALANCE (CAN BE ZERO I.E. EXISTING ONLY)
total_balance = sizing + (float(web3.from_wei(agent_balance, 'ether')) / token_price)

print("ISOLATED POSITION: ", round(sizing * profitability, 3), "$VIRTUAL")
print("TOTAL POSITION: ", round(total_balance * profitability, 3), "$VIRTUAL")